[Accueil](../../index.ipynb) > [Sommaire Première](../index.ipynb)

# TP système solaire

Le but de ce TP est de construire un petit site web qui va présenter l'ensemble des planètes de notre système solaire.

Afin de le réaliser nous allons utiliser un framework web écrit en Python qui se nomme [Flask](https://flask.palletsprojects.com/en/2.3.x/).

## Mise en place

- Créez un python virtuel que vous nommerez *flask*.
- Allez sur https://github.com/saintlouis29/simple_web et faites un fork de ce projet.
- Depuis le terminal, clonez votre dépôt "forké" sur votre ordinateur (dans *projets* par exemple)
- Depuis le terminal, allez dans le dossier cloné et lancez la commande *python3 setup.py develop*
- Allez sur votre IDE (Codium par exemple) et selectionnez le python virtuel précedemment créé.
- Ouvrir le fichier app.py
- Executez le fichier
- Allez sur http://localhost:5000

Si tout c'est passé vous devriez voir *Hello World !*.

## Observations des paquets http sur Wireshark

- Lancez le programme Wireshark

Le but est d'écouter sur l'interface  loopback (c'est à dire uniquement le réseau interne de votre machine) et de se limiter au protocole http sur le port 5000.

- Allez dans Capture > Options et selectionnez uniquement l'interface loopback
- Dans la partie filtre, ajoutez *tcp.port == 5000 and http* et appliquez ce filtre

Si tout s'est bien passé, si vous faite une requête sur http://localhost:5000, wireshark devrait capture la requête du navigateur et la réponse du serveur web.

**Observer le contenu des paquets html**

## utilisation des templates de Flask

### Création d'un template

Il serait très fastidieux et peu pratique de coder le html de notre page directement dans le fichier app.py.

Flask possède un système de *templates* qui permet de coder le html dans des fichiers séparés.

- Créez un dossier *templates* (il doit se nommer ainsi)
- Dans ce dossier, créez un fichier index.html
- Copiez-collez le code suivant dans votre fichier app.py

```python
from flask import Flask, render_template
app = Flask(__name__)

@app.route("/")
def accueil():
    return render_template("index.html")

# en mode debug, le serveur raffraichit automatiquement les modifications apportées au code
app.run(debug=True)

```

- Copiez-collez le code html suivant dans le fichier index.html

```html
<!DOCTYPE html>
<html lang="fr">
  <head>
    <meta charset="utf-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
    <title>Le TP web</title>
    <meta name="viewport" content="width=device-width, initial-scale=1" />
  </head>
  <body>
    <h1>Application sur le système solaire</h1>
    <p>Voici une petite application sur le système solaire.</p>

    <p>Les 3 planètes du système solaire sont :</p>
    <ol>
      <li>Mercury</li>
      <li>Venus</li>
      <li>Earth</li>
    </ol>
  </body>
</html>

```
Vous deviez obtenir quelque chose comme cela:

![Capture1](img/Screenshot_TP_web1.png)

**Explications:**

La fonction *render_template* permet de générer le fichier index.html lorsque la racine '/' du site est demandée.

### Les templates : une ouverture sur le html dynamique

Nous faisons une application web, nous n'allons donc pas coder entièrement le code html pour afficher toutes les planètes du système solaire.

Dans le fichier *app.py* ajouter la fonction suivante:

```python
def get_planets():
    """Return an ordered list of the planets of the solar system
    Each planet is a dictionnary.
    """
    return [
        {"planet": "Mercury"},
        {"planet": "Venus"},
        {"planet": "Earth"},
        {"planet": "Mars"},
        {"planet": "Jupiter"},
        {"planet": "Saturn"},
        {"planet": "Uranus"},
        {"planet": "Neptune"},
        {"planet": "Pluto"},
    ]
```

Dans la fonction *accueil*, nous allons passer une variable *planets* qui vaut ce que retourne la fonction *get_planets*.

Cela donne:

```python
@app.route("/")
def accueil():
    return render_template("index.html", planets=get_planets())
```

Les templates de Flask utilisent le langage [Jinja ](https://jinja.palletsprojects.com/en/3.1.x/templates/) qui permet entre autre:
- d'afficher des variables
- de faire des boucles
- d'inclure des conditions

L'idée est de faire une **boucle** sur la variable *planets*, puis pour chaque planète, afficher la valeur de la clé *planet*.

Voici la liste des planètes générée depuis la variable *planets* dans notre fichier *index.html*

```html
    <ol>
      {% for p in planets %}
      <li>{{ p.planet }}</li>
      {% endfor %}
    </ol>
```

On peut également afficher dynamiquement le nombre de planètes:
```html
<p>Les {{planets|length}} planètes du système solaire sont :</p>
```

<div class="alert alert-info">
    <p>Il faut bien assimiler que la page html a été générée <strong>coté serveur</strong>. Le serveur web (Flask) a reçu une requete du client et a répondu en générant une page web.</p>
    <p>Pour s'en convaincre:</p>
    <ul>
        <li>regarder sur wireshark la réponse reçue par votre navigateur.</li>
        <li>Inspecter la page de votre navigateur</li>
    </ul>
</div>

## Une page pour chaque planète

Nous allons créer une page dédiée à chaque planète du systeme solaire.

Evidemment nous n'allons pas créer chaque page individuellement !!!

Le concept est le suivant:

Nous allons créer **une** page html générique que nous nommons, dans le dossier **templates**, *planet.html*.

- Depuis la page d'accueil, nous allons ajouter des liens vers cette page en passant en paramètre dans l'url l'id de la planète. Ainsi le lien vers la planète Mars doit avoir l'url suivante: http://127.0.0.1:5000/planet?planet_id=Mars

- Dans le fichier app.py nous allons ajouter la nouvelle route */planet* 
- La fonction planet(), associée à cette route va:
  - Récupérer le *planet_id* de la planète depuis la requête;
  - Appeler une fonction *get_planet_from_id(planet_id)* qui retournera les infos de la planète sous forme de dictionnaire;
  - Rendre le template planet.html en passant comme variable la planète trouvée précédemment.

Procédons donc dans l'ordre:

- Ajout des liens dans le fichier index.html
  - Modifier le code pour ajouter les liens 
  ```html
  <li><a href="planet?planet_id={{ p.planet }}">{{ p.planet }}</a></li>
  ```
  - Vérifier que les urls des liens sont correctement générés
  
- Dans le fichier app.py:
  - en début de fichier ajouter l'import de *request*;
  - ajouter la fonction *get_planet_from_id(planet_id)*

```python
    def get_planet_from_id(planet_id):
        """Return a dictionnary of the planet corresponding to the planet_id.
        If the planet is not found, return an empty dictionnary

        Args:
            planet_id (string): The id of the planet.
        """
        for p in get_planets():
            if p["planet"] == planet_id:
                return p
        return {}
```
  - ajouter la route */planet* et la fonction associée

```python
    @app.route("/planet", methods=['GET'])
    def planet():
        # On récupère l'id passé en paramètre
        planet_id = request.args.get("planet_id")
        planet = get_planet_from_id(planet_id)
        return render_template("planet.html", planet=planet)

```
  
- Ajouter, dans le dossier *templates*, le fichier planet.html
- Ajouter ce code html dans le fichier planet.html


```html
<!DOCTYPE html>
<html lang="fr">
  <head>
    <meta charset="utf-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
    <title>{{ planet.planet}}</title>
    <meta name="viewport" content="width=device-width, initial-scale=1" />
  </head>
  <body>
    <h1><a href="/">Accueil</a></h1>
    <h2>{{ planet.planet}}</h2>
    <p>Voici la page de la planète {{ planet.planet }}.</p>
    <p><a href="/">Retour à l'accueil</a></p>
  </body>
</html>

```

Si tout se déroule comme prévu, vous devriez obtenir à peu près ceci:

![Capture2](img/Screenshot_TP_web2.png)

<div class="alert alert-info">Dans Wireshark, observez le paramètre planet_id qui est passé en paramètre lors de la requête vers le serveur.</div>

![Capture3](img/Screenshot_TP_web3.png)

### Premier travail à faire

Les données sur les planètes du système solaire sont disponibles en fichier csv sur à l'adresse suivante:https://devstronomy.com/#/datasets

- Télécharger le fichier planets.csv dans votre application
- Modifier la fonction get_planets afin qu'elle retourne une liste de dictionnaires en utilisant ce fichier csv
- Modifier la page *planet.html* afin d'afficher, sous forme de [tableau](https://developer.mozilla.org/fr/docs/Web/HTML/Element/table), les informations suivantes:
  - Masse (1024kg)
  - Diamètre (km)
  - Densité (kg/m3)
  - Gravité (m/s2)


## Un peu de cosmétique

Dans cette première partie du TP nous avons généré des pages html. Les pages sont sémantiquement correctes mais nécessitent un peu de cosmétique pour les rendre plus **sexy**.

C'est maintenant qu'entre en scène le **CSS**.

Flask permet de rendre des fichiers statiques (qui ne changent pas), c'est le cas des fichiers css.

Pour ce faire:
  - Créer un dossier nommé *static*;
  - Y ajouter un fichier nommé *style.css*
  - Y ajouter les lignes suivantes:
  
```css

html{
    background-color:greenyellow;
    padding:50px;
}

body{
    background-color:white;
    border:1px solid black;
    padding: 10px;
    border-radius: 10px;
}

```

  - Dans le \<head\> des fichiers *index.html* et *planet.html* ajouter le code suivant qui permet d'appeler le fichier *style.css*.

```html

    <link
      rel="stylesheet"
      type="text/css"
      href="{{ url_for('static', filename='style.css') }}"
    />

```

Allez sur le site, la présentation devrait ressembler à ceci:

![Capture4](img/Screenshot_TP_web4.png)
  
<div class="alert alert-info">
    <p>Il est indispensable de comprendre que le serveur a généré une page web incluant un lien vers un fichier css. Le client a requété l'URL de ce CSS et l'a interprété.</p>
    <p>L'interpréation du CSS est effectuée par le client (le navigateur).</p>
</div>

<div class="alert alert-warning">Mes talents de designer web s'arrêtent là. Coder du css est un vrai métier, et ce n'est pas non plus dans l'esprit de la spécialité NSI. Nous allons donc utiliser des frameworks CSS pour nous faciliter la tâche.</div>

Lorsqu'on veut un design web aux petits onions, il est nécessaire de faire appel à un designer web qui va, sur mesure, coder le css du site.

Cependant il existe des solutions clés en main qui permettent d'obtenir un rendu acceptable pour une application web : il s'agit des **frameworks CSS**.

Il en existe une [multitude](https://mobiskill.fr/blog/conseils-emploi-tech/quels-sont-les-meilleurs-frameworks-css-en-front-end/) dont voici une courte liste:
 - [Bootstrap](https://getbootstrap.com/) : développé par Twitter
 - [Foundation](https://get.foundation/)
 - [Materialize](https://materializecss.com/) : developpé par Google

Ces frameworks offrent de nombreux avantages:

- Obtenir un design acceptable sans être designer;
- Etre compatible avec l'ensemble des navigateurs modernes;
- Permettre aisement le *responsive design* : adapter le rendu aux différents supports (tablette, ecran PC, smartphone, papier...)
- ...

### Deuxième travail à faire

- Intégrer [Bootstrap](https://getbootstrap.com/docs/5.3/getting-started/introduction/) à votre site en utilisant un CDN (content Delivery Network)
- Commencer par intégrer un [container](https://getbootstrap.com/docs/5.3/layout/containers/);
- Essayer, par exemple d'ajouter un [breadcrumb](https://getbootstrap.com/docs/5.3/components/breadcrumb/) dans vos pages